In [1]:
from music21 import *
from matplotlib import pyplot as plt
from sklearn import svm, naive_bayes, neighbors
import numpy as np
import data_loader as loader

In [2]:
%%capture
composerNames = ["debussy","tchaikovsky","mozart","victoria","beethoven"]
#count = min(loader.get_data_counts(composerNames))
x_train, x_test,y_train, y_test = loader.load("chords_t",composerNames)


In [3]:
print("train",loader.get_data_counts(y=y_train,composer_names=composerNames))
print("test",loader.get_data_counts(y=y_test,composer_names=composerNames))

train {'debussy': 119, 'tchaikovsky': 165, 'mozart': 423, 'victoria': 188, 'beethoven': 380}
test {'debussy': 40, 'tchaikovsky': 54, 'mozart': 124, 'victoria': 61, 'beethoven': 61}


In [ ]:
classes = list(set(y_test))
test_data = list(zip(x_test,y_test))
for label in classes:
    label_data = [tupple[0] for tupple in test_data if tupple[1] == label][:10]
    plt.plot(label_data)
    plt.xlabel("Examples")
    plt.show()

In [14]:
classifiers = [svm.SVC(kernel="poly"),
               neighbors.KNeighborsClassifier(n_neighbors=50),
               naive_bayes.MultinomialNB()]

for clf in classifiers:
    clf.fit(x_train,y_train)
    print(clf)
    print(f"acc: {clf.score(x_test,y_test)}","\n")

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='poly',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)
acc: 0.4147058823529412 

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=50, p=2,
           weights='uniform')
acc: 0.4176470588235294 

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)
acc: 0.40588235294117647 



In [15]:
for i in range(1,60):
    k_nn = neighbors.KNeighborsClassifier(n_neighbors=i)
    k_nn.fit(x_train, y_train)
    print(f"k = {i}, acc = {k_nn.score(x_test,y_test)}")

k = 1, acc = 0.21764705882352942
k = 2, acc = 0.18235294117647058
k = 3, acc = 0.2
k = 4, acc = 0.2323529411764706
k = 5, acc = 0.23823529411764705
k = 6, acc = 0.23823529411764705
k = 7, acc = 0.2529411764705882
k = 8, acc = 0.2823529411764706
k = 9, acc = 0.3058823529411765
k = 10, acc = 0.3029411764705882
k = 11, acc = 0.3088235294117647
k = 12, acc = 0.36176470588235293
k = 13, acc = 0.36176470588235293
k = 14, acc = 0.3558823529411765
k = 15, acc = 0.3735294117647059
k = 16, acc = 0.37941176470588234
k = 17, acc = 0.38235294117647056
k = 18, acc = 0.38823529411764707
k = 19, acc = 0.38235294117647056
k = 20, acc = 0.3941176470588235
k = 21, acc = 0.40588235294117647
k = 22, acc = 0.3941176470588235
k = 23, acc = 0.40294117647058825
k = 24, acc = 0.4323529411764706
k = 25, acc = 0.4264705882352941
k = 26, acc = 0.43529411764705883
k = 27, acc = 0.4411764705882353
k = 28, acc = 0.43823529411764706
k = 29, acc = 0.4441176470588235
k = 30, acc = 0.43823529411764706
k = 31, acc = 0.438